# FLAT input data prepping

This code prepares the final input file to the FLAT model. 

**Original code:** [Konstantinos Pegios](https://github.com/kopegios) <br />
**Conceptualization & Methodological review :** [Alexandros Korkovelos](https://github.com/akorkovelos) & [Konstantinos Pegios](https://github.com/kopegios)<br />
**Updates, Modifications:** [Alexandros Korkovelos](https://github.com/akorkovelos)<br />
**Funding:** The World Bank (contract number: 7190531), [KTH](https://www.kth.se/en/itm/inst/energiteknik/forskning/desa/welcome-to-the-unit-of-energy-systems-analysis-kth-desa-1.197296)

In [ ]:
#Import modules and libraries

import geopandas as gpd
import rasterio as rio
import pandas as pd
import fiona
import gdal
import osr
import ogr
import rasterio.mask
import time
import os
import ogr, gdal, osr, os
import numpy as np
import itertools
import re

from osgeo import gdal,ogr
import struct
import csv

import glob

from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling
from rasterstats import point_query
from pyproj import Proj
from shapely.geometry import Point, Polygon

In [ ]:
# Import data 

#path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data"
name_of_file = "Ben_Maize_Harv_2000_admin2_1km.csv"

In [ ]:
# Import csv as pandas dataframe
df = pd.read_csv(path + "\\" + name_of_file)

In [ ]:
#df = df.drop(["yield", "production_ha"], axis=1)
df.rename(columns={'id': 'alloc_key'}, inplace=True)

In [ ]:
df.rename(columns={'COUNTRY_COUNTRY': 'state'}, inplace=True)
df.rename(columns={'X_deg': 'lon'}, inplace=True)
df.rename(columns={'Y_deg': 'lat'}, inplace=True)
df.rename(columns={'area_ha': 'statearea_ha'}, inplace=True)
df.rename(columns={'country_code': 'c_code'}, inplace=True)

In [ ]:
df.head()

In [ ]:
#Function to change the order of columns in the dataframe 
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

In [ ]:
# Allocate order in dataframe columns
df = change_column_order(df, 'alloc_key', 1)
df = change_column_order(df, 'lon', 2)
df = change_column_order(df, 'lat', 3)
df = change_column_order(df, 'country', 4)
df = change_column_order(df, 'c_code', 5)
df = change_column_order(df, 'state', 6)
df = change_column_order(df, 'statearea_ha', 7)
df = change_column_order(df, 'perimeter_km', 8)
df = change_column_order(df, 'year', 9)
df = change_column_order(df, 'crop', 10)
df = change_column_order(df, 'harv_area_ha', 11)

In [ ]:
df.head()

In [ ]:
# Add geometry and convert to spatial dataframe in source CRS
df['geometry'] = list(zip(df['lon'], df['lat']))
df['geometry'] = df['geometry'].apply(Point)
df = gpd.GeoDataFrame(df, geometry='geometry', crs={'init': 'epsg:4326'})

# Reproject data in to Ordnance Survey GB coordinates
df_osgb = df.to_crs({'init': 'epsg:32631'})

In [ ]:
df_osgb.head()

<div class="alert alert-block alert-warning">
<b>Note:</b> The geodataframe, contains crop points with needed attributes including a geometry column. It needs to be exported as shapefile. However, when this is done with geopandas, the column geometry (Point(x,y)) is not transfered in the attribute table. This is however needed for the extraction process below, otherwise "None" values are returned. In order to overcome this issue, the csv is imported into Qgis and then exported as feature layer in the same folder. This solves the issue. 
</div>

In [ ]:
# Define output path
# Overwriting the csv file
#path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data"
shpname = "Ben_Maize_Harv_2000_admin2_1km_new.csv"

#MAIZE
df_osgb.to_csv(os.path.join(path,"{c}".format(c=shpname)), index=False)
df_osgb.to_file(os.path.join(path,"{c}.shp".format(c="Ben_Maize_2018_admin2_new")))
# Define output path
#path = r"N:\Agrodem\Downscaling\Output_Data\Crop_Maps"
#path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data\Crop_Maps"

#Maize


#### >>> Please open Qgis and act as suggested above<<<

## Extracting raster input to csv 

In [ ]:
#raster_path = r"N:\Agrodem\Downscaling\Input_Data\rasters"
raster_path = r"C:\Oluchi\Downscaling\Input_Data\raster"
raster_files =[]
#csvoutpath = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input\1km_Rice"
csvoutpath = r"C:\Oluchi\Downscaling\Output_Data\FLAT_Input\1km_Maize"

print ("Reading independent variables...")

for i in os.listdir(raster_path):
    if i.endswith('.tif'):
        raster_files.append(i)  

#shp_filename = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops\shapefiles\Moz_Rice_Harv_2000_admin2_1km.shp"
shp_filename = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data\Ben_Maize_2018_admin2_new.shp"
print ("Extracting raster values to points...")

for i in raster_files:
    print("Extracting " + i + " values...")
    src_filename = raster_path + "\\" + i 
    li_values = list()

    src_ds=gdal.Open(src_filename) 
    gt=src_ds.GetGeoTransform()
    rb=src_ds.GetRasterBand(1)
    ds=ogr.Open(shp_filename)    
    lyr=ds.GetLayer()
    
    for feat in lyr:
        geom = feat.GetGeometryRef()
        feat_id = feat.GetField('alloc_key')
        mx,my=geom.GetX(), geom.GetY()  #coord in map units

        #Convert from map to pixel coordinates.
        #Only works for geotransforms with no rotation.
        px = int((mx - gt[0]) / gt[1]) #x pixel
        py = int((my - gt[3]) / gt[5]) #y pixel


        intval=rb.ReadAsArray()
        li_values.append([feat_id, intval[0]])
        
    print ("Writing " + i + " values to csv...")
    
    with open(csvoutpath + "\\" + i.split('.')[0] + '.csv', 'w') as csvfile:
        wr = csv.writer(csvfile)
        wr.writerows(li_values)

## Merge csv files with crop

In [ ]:
#Import data 

#path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data"
name_of_file = "Ben_Maize_Harv_2000_admin2_1km.csv"

flat_input = pd.read_csv(path + "\\" + name_of_file)

In [ ]:
csvoutpath = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input\1km_Rice"

csv_files = []

print ("Reading csv files...")

for i in os.listdir(csvoutpath):
    if i.endswith('.csv'):
        csv_files.append(i)  

for i in csv_files:
    print('Reading...'+ i)    
    df_csv = pd.read_csv(csvoutpath + "//" + i, index_col=None, header=None)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].astype(str)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace('[','')
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace(']','')
    columnName = i.split('.')[0]
    print("Merging..." + columnName)
    flat_input[columnName] = df_csv.iloc[:,1]

## Cleaning and normalizing dataframe

In [ ]:
#Show columns
flat_input.columns

In [ ]:
#Delete geometry column
flat_input = flat_input.drop(["geometry"], axis=1)

#### Convert values in newly added columns into float 

In [ ]:
flat_input["harv_area_ha"] = flat_input["harv_area_ha"].astype(float)
flat_input["BDRICM"] = flat_input["BDRICM"].astype(float)
flat_input["BLD"] = flat_input["BLD"].astype(float)
flat_input["CLYPPT"] = flat_input["CLYPPT"].astype(float)
flat_input["DRAINFAO"] = flat_input["DRAINFAO"].astype(float)
flat_input["elevation"] = flat_input["elevation"].astype(float)
flat_input["EVI"] = flat_input["EVI"].astype(float)
flat_input["Landcover30m"] = flat_input["Landcover30m"].astype(float)
flat_input["NDVI"] = flat_input["NDVI"].astype(float)
flat_input["ORCDRC"] = flat_input["ORCDRC"].astype(float)
flat_input["PHIHOX"] = flat_input["PHIHOX"].astype(float)
flat_input["prec"] = flat_input["prec"].astype(float)
flat_input["slope"] = flat_input["slope"].astype(float)
flat_input["srad"] = flat_input["srad"].astype(float)
flat_input["tavg"] = flat_input["tavg"].astype(float)
flat_input["TEXMHT"] = flat_input["TEXMHT"].astype(float)
flat_input["wind"] = flat_input["wind"].astype(float)

In [ ]:
flat_input.dtypes

#### Normalize EVI and NDVI 

In [ ]:
flat_input['EVI'] = flat_input['EVI']*0.0001
flat_input['NDVI'] = flat_input['NDVI']*0.0001

#### Keep areas that are indicated as cropland in GFSAD30m

In [ ]:
flat_input = flat_input[flat_input["Landcover30m"] == 2]
flat_input = flat_input.drop(["Landcover30m"], axis=1)

#### Fixing out-of-range values

In [ ]:
# remove blank spaces in state names    
flat_input["state"].replace('\s+', '_',regex=True,inplace=True)
flat_input["state"].replace("'", '_',regex=True,inplace=True)

In [ ]:
# Re-indexing allocation keys to avoid dublicates
flat_input = flat_input.assign(alloc_key=np.arange(len(flat_input))).reset_index(drop=True)

In [ ]:
# Turning NaN rows to 0
flat_input.fillna(0,inplace=True)

In [ ]:
# Turn all negative values to 0
pred_columns = ['tavg', 'srad', 'prec', 'wind', 'PHIHOX',
                'BDRICM', 'BLD', 'CLYPPT', 'TEXMHT', 'ORCDRC',
                'DRAINFAO', 'slope', 'elevation']

for col in pred_columns:
    mvalue = flat_input[col].mean()
    print (mvalue)
    flat_input[col][flat_input[col]<0] = 0

In [ ]:
for col in pred_columns:
    mvalue = flat_input[col].mean()
    print (mvalue)

In [ ]:
flat_input.columns

In [ ]:
flat_input = change_column_order(flat_input, 'alloc_key', 0)
flat_input = change_column_order(flat_input, 'state', 1)
flat_input = change_column_order(flat_input, 'lon', 2)
flat_input = change_column_order(flat_input, 'lat', 3)
flat_input = change_column_order(flat_input, 'country', 4)
flat_input = change_column_order(flat_input, 'c_code', 5)

flat_input = change_column_order(flat_input, 'statearea_ha', 6)
flat_input = change_column_order(flat_input, 'perimeter_km', 7)
flat_input = change_column_order(flat_input, 'year', 8)
flat_input = change_column_order(flat_input, 'crop', 9)
flat_input = change_column_order(flat_input, 'harv_area_ha', 10)
flat_input = change_column_order(flat_input, 'yield', 11)
flat_input = change_column_order(flat_input, 'production_ha', 12)

flat_input = change_column_order(flat_input, 'tavg', 13)
flat_input = change_column_order(flat_input, 'srad', 14)
flat_input = change_column_order(flat_input, 'prec', 15)
flat_input = change_column_order(flat_input, 'wind', 16)

flat_input = change_column_order(flat_input, 'PHIHOX', 17)
flat_input = change_column_order(flat_input, 'BDRICM', 18)
flat_input = change_column_order(flat_input, 'BLD', 19)
flat_input = change_column_order(flat_input, 'CLYPPT', 20)
flat_input = change_column_order(flat_input, 'TEXMHT', 21)
flat_input = change_column_order(flat_input, 'ORCDRC', 22)
flat_input = change_column_order(flat_input, 'DRAINFAO', 23)

flat_input = change_column_order(flat_input, 'EVI', 24)
flat_input = change_column_order(flat_input, 'NDVI', 25)

flat_input = change_column_order(flat_input, 'slope', 26)
flat_input = change_column_order(flat_input, 'elevation', 27)

In [ ]:
flat_input.head()

In [ ]:
flat_input.columns

#### Final column fixing

In [ ]:
# give crop name
crop_modelled ="Rice"

# Dropping columns
flat_input = flat_input.drop(["country"], axis=1)
flat_input = flat_input.drop(["c_code"], axis=1)
flat_input = flat_input.drop(["perimeter_km"], axis=1)
flat_input = flat_input.drop(["year"], axis=1)
flat_input = flat_input.drop(["crop"], axis=1)
flat_input = flat_input.drop(["yield"], axis=1)
flat_input = flat_input.drop(["production_ha"], axis=1)


#Remaning columns
flat_input.rename(columns={'state': 'NAME'}, inplace=True)
flat_input.rename(columns={'statearea_ha': 'statearea'}, inplace=True)
flat_input.rename(columns={'harv_area_ha': crop_modelled}, inplace=True)


In [ ]:
flat_input

In [ ]:
# This part prints full results

path = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input"
name_of_flat_input_file = "flat_input_Rice_1km"

flat_input.to_csv(os.path.join(path,"{c}.csv".format(c=name_of_flat_input_file)), index=False)